# 1. Actualizar e Instalar librerias

In [ ]:
!pip install --upgrade scikit-learn
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 51.9 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.9 MB/s eta 0:00:00


# 2. Librerias y Carga de Datos

In [ ]:
# utilizar: !pip install category_encoders (Si se requiere target encoding, para este proyecto no es necesario)
# utilizar: !pip install --upgrade scikit-learn (Actualizar Sklearn SI es necesario para este proyecto)

# Para ler archivos json
import json
# Para guardar modelos y metadatos de modelo
import joblib
# Para Medir Tiempos
import time

# Para acceder a mi drive y los archivos
from google.colab import drive

# Manejo de datos
import pandas as pd
import numpy as np

# Para producir tuplas con diferentes combinaciones
from itertools import product

# Metricas de evaluacion de modelos
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# Classes para buscar mejores hiperparametros en un SeriesTiempo
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

# RandomForestRegressor Model
from sklearn.ensemble import RandomForestRegressor
# CatBoost Regression Model
from catboost import CatBoostRegressor
# XGBoostRegressor Model
from xgboost import XGBRegressor
# LightGBM Regression Model
from lightgbm import LGBMRegressor

In [ ]:
drive.mount('/content/gdrive',force_remount=True)
ruta_2='/content/gdrive/MyDrive/Colab Notebooks/TABLAS/providencia_ventas_historial.json'

Mounted at /content/gdrive


# 3. Ingenieria de datos

In [ ]:
# 1. leer los datos (ventas)
df_venta = pd.read_json(ruta_2)

df_copia = df_venta.copy()

# 1. Guardar los productos, anhos, y meses unicos, como un Series: (Se usaran para crear el esqueleto mas adelante para productos vacios)
productos = df_copia['Producto'].unique()
anhos = df_copia['Year'].unique()
meses = df_copia['Month'].unique()

# 2. Solo necesitamos las fechas, los productos y las ventas del DF venta:
df_copia = df_copia[['Year','Month','Day','Producto','Cantidad']]
# Tambien eliminar el mes 09 porque esta incompleto:
df_copia = df_copia[~(df_copia['Month']==9)]
# Y el primer mes del 2023 por estar incompleto
df_copia = df_copia[~((df_copia['Month']==1) & (df_copia['Year']==2023))]
display(df_copia.head(1))

# 3. Necesitamos agrupar por anho,mes,producto, el dia no es necesario, aplicar funcion suma:
grupo = df_copia.groupby(by=['Year','Month','Producto'])['Cantidad'].sum()
df_grupo = pd.DataFrame(grupo)
df_grupo = df_grupo.reset_index()
display(df_grupo.head(1))

,Year,Month,Day,Producto,Cantidad
2699,2023,2,1,CARTUCHO SILICON PEPE TRANSP.,2.0


,Year,Month,Producto,Cantidad
0,2023,2,ABRAZADERA FOFO DE 3,1.0


In [ ]:
# Combinar todos los posibles anho, mes, producto del esqueleto, (Este paso es importante para que los productos se alineen al aplicar lags.)

# 5. Crear el esqueleto completo de todas las combinaciones de (Año, Mes, Producto):
esqueleto_completo = list(product(anhos, meses, productos))

# 6. Convertir la lista de tuplas en un DataFrame:
df_esqueleto = pd.DataFrame(esqueleto_completo, columns=['Year', 'Month', 'Producto'])

# IMPORTANTE (Este esqueleto contienen todas las combinaciones posibles, pero debemos quitar Enero 2023, y a partir de Septiembre 2025):
# Primero Enero 2023:
df_esqueleto = df_esqueleto[~((df_esqueleto['Year']==2023) & (df_esqueleto['Month']==1))]
# A partir de Septiembre 2025
meses_no_existentes = [9,10,11,12]
df_esqueleto = df_esqueleto[~((df_esqueleto['Year']==2025) & (df_esqueleto['Month'].isin(meses_no_existentes)))]
df_esqueleto = df_esqueleto.reset_index(drop=1)

# Muestra un ejemplo del esqueleto
display(df_esqueleto.head(1))
display(df_esqueleto.tail(1))
print()
print(f"Número de filas del esqueleto: {len(df_esqueleto)}")
print(f"Número de filas de las ventas agrupadas: {len(df_grupo)}")

,Year,Month,Producto
0,2023,2,CODO DE LAMINA 90 X 4


,Year,Month,Producto
82273,2025,8,QUEMADOR ESTRELLA 15 CM



Número de filas del esqueleto: 82274
Número de filas de las ventas agrupadas: 24365


In [ ]:
# 7. Fusionar el DataFrame de ventas agrupadas con el esqueleto:
# Usamos un 'left' merge para mantener todas las combinaciones del esqueleto.
df_final = pd.merge(df_esqueleto,df_grupo,on=['Year', 'Month', 'Producto'],how='left')

# El esqueleto contendra productos no vendidos mensualmente para cada mes, por ende la fusion tendra valores NaN
# 8. Rellenar los valores NaN (ventas faltantes) con 0:
df_final['Cantidad'] = df_final['Cantidad'].fillna(0)

# Muestra las primeras filas del DataFrame final (debería mostrar algunas cantidades 0)
display(df_final.head(1))
display(df_final.tail(1))
print(f"Número de filas del DataFrame final (debe coincidir con el esqueleto): {len(df_final)}")

,Year,Month,Producto,Cantidad
0,2023,2,CODO DE LAMINA 90 X 4,3.0


,Year,Month,Producto,Cantidad
82273,2025,8,QUEMADOR ESTRELLA 15 CM,0.0


Número de filas del DataFrame final (debe coincidir con el esqueleto): 82274


In [ ]:
# 9 Obtener la columna fecha de tipo (datetime64) para poder aplicar 3 lags y una media movil con un suavizado de 3 meses.
df_final['Fecha'] = df_final['Year'].astype(str)+'-'+df_final['Month'].astype(str)
df_final['Fecha'] = pd.to_datetime(df_final['Fecha'],format='%Y-%m')

df_fecha = df_final.copy()
df_fecha = df_fecha[['Fecha','Producto','Cantidad']]
display(df_fecha.head(1))
display(df_fecha.tail(1))
print(f'Dimension del DataFrame Fechas {df_fecha.shape}')

,Fecha,Producto,Cantidad
0,2023-02-01,CODO DE LAMINA 90 X 4,3.0


,Fecha,Producto,Cantidad
82273,2025-08-01,QUEMADOR ESTRELLA 15 CM,0.0


Dimension del DataFrame Fechas (82274, 3)


In [ ]:
# Creare una funcion que establezca las variables lag y media movil:
def lags_movil_corregido(df, max_lag, rolling_window):
  # 0. agrego las columnas de fecha:
  df['Month'] = df['Fecha'].dt.month
  df['Year'] = df['Fecha'].dt.year # El año se usara mas adelante para crear la particion entrenamiento prueba
  # pero el la funcion de limpieza automatica no es necesario incluirla.

  # 1. Aplicar Lag por grupo de Producto:
  for lag in range(1, max_lag + 1):
    df[f'Cantidad_lag_{lag}'] = (df.groupby('Producto')['Cantidad'].shift(lag))

  # 2. Aplicar Media Móvil por grupo de Producto:
  # 1. Elimina el índice 'Producto' temporalmente, Aplicar el desplazamiento al resultado de la media móvil
  df['Rolling_mean'] = (df.groupby('Producto')['Cantidad'].rolling(window=rolling_window).mean().shift(1).reset_index(level=0, drop=True))

  return df

# Aplicar la función al DataFrame final:
datos = lags_movil_corregido(df_fecha, 3, 3)

# Muestra las filas donde el mismo producto cambia de mes:
print("Ejemplo de Lag (debe verse el valor del mes anterior del mismo producto):")
display(datos[datos['Producto'] == datos['Producto'].iloc[5]].head(4))
print(f'Dimension del DataFrame final: {datos.shape}')

# A pesar de que el lag es de 3 meses, se necesitan 4 meses para cualquier nuevo data, porque es el que recibe la informacion de los 3 anteriores: verlo en la tabla de abajo:

Ejemplo de Lag (debe verse el valor del mes anterior del mismo producto):


,Fecha,Producto,Cantidad,Month,Year,Cantidad_lag_1,Cantidad_lag_2,Cantidad_lag_3,Rolling_mean
5,2023-02-01,CASQUILLO DE LAMINA 4,4.0,2,2023,NaN,NaN,NaN,1.000000
2659,2023-03-01,CASQUILLO DE LAMINA 4,0.0,3,2023,4.0,NaN,NaN,NaN
5313,2023-04-01,CASQUILLO DE LAMINA 4,0.0,4,2023,0.0,4.0,NaN,NaN
7967,2023-05-01,CASQUILLO DE LAMINA 4,0.0,5,2023,0.0,0.0,4.0,1.333333


Dimension del DataFrame final: (82274, 9)


# 4. Preparacion de datos para el modelo (Variables y Objetivo)

In [ ]:
# Preparacion de datos para el modelo:

# Preparamos el encoding para la categoria de meses, el resto de variables estan en la misma escala numerica, pueden dejarse igual.

# Sen/Cos para codificar los meses de manera ciclica
P = 12 # <- El periodo son los 12 meses del anho
# Crear las dos columnas ciclicas:
datos['Mes_seno'] = np.sin((2 * np.pi * datos['Month'])/ P)
datos['Mes_coseno'] = np.cos((2 * np.pi * datos['Month']) / P)

info = datos.copy()

# Division entrenamiento y prueba
train = info[~((info['Year']==2025) & (info['Month'].isin([8])))]
test = info[((info['Year']==2025) & (info['Month'].isin([8])))]

print(f'Dimension de los datos de entrenamiento: {train.shape}')
print(f'Dimension de los datos de prueba: {test.shape}')

# Eliminar ahora si todas las filas con valores NaN resultantes de los lags
train = train.dropna()

train = train.reset_index(drop=1)
test = test.reset_index(drop=1)
print()
print(f'Dimension de entrenamiento despues de limpieza {train.shape}')
print()
# Definimos la matriz X y el vector Y de respuestas:
productos_en_train = train['Producto'].copy()
x_train = train.drop(labels=['Fecha','Producto','Cantidad','Year','Month'],axis=1)
y_train = train['Cantidad']

productos_en_test = test['Producto'].copy()
x_test = test.drop(labels=['Fecha','Producto','Cantidad','Year','Month'],axis=1)
y_test = test['Cantidad']

display(x_train.head(1))
display(x_test.head(1))


Dimension de los datos de entrenamiento: (79620, 11)
Dimension de los datos de prueba: (2654, 11)

Dimension de entrenamiento despues de limpieza (71658, 11)



,Cantidad_lag_1,Cantidad_lag_2,Cantidad_lag_3,Rolling_mean,Mes_seno,Mes_coseno
0,0.0,0.0,3.0,1.0,0.5,-0.866025


,Cantidad_lag_1,Cantidad_lag_2,Cantidad_lag_3,Rolling_mean,Mes_seno,Mes_coseno
0,7.0,3.0,3.0,4.333333,-0.866025,-0.5


# 5. Modelo Dummy (Baseline)

In [ ]:
# Modelo dummy
X_tn = train.copy()
X_ts = test.copy()

# Solo necesitamos el ultimo mes de train y el primero de prueba:
X_julio = X_tn[(X_tn['Month']==7) & (X_tn['Year']==2025)]
X_agosto = X_ts[(X_ts['Month']==8) & (X_ts['Year']==2025)]

# 1. ¿Tienen la misma longitud?
print(f"Filas en Mayo: {len(X_julio)}")
print(f"Filas en Junio: {len(X_agosto)}")
print()

# 2. Revisa si el orden por ID es el mismo al inicio
print("\n--- Cabecera Julio ---")
display(X_julio[['Producto', 'Cantidad']].head(1))
print("\n--- Cabecera Junio ---")
display(X_agosto[['Producto', 'Cantidad']].head(1))
print()

prediccion_dummy = X_julio['Cantidad'].copy()
respuestas_dummy = X_agosto['Cantidad'].copy()

error_cuadratico_dummy = np.sqrt(mean_squared_error(respuestas_dummy,prediccion_dummy))
error_dummy = mean_absolute_error(respuestas_dummy,prediccion_dummy)
valor_r2_dummy = r2_score(respuestas_dummy,prediccion_dummy)

print(f'Error cuadratico del modelo dummy: {error_cuadratico_dummy}')
print(f'Error absoluto del modelo dummy: {error_dummy}')
print(f'Valor R2 del modelo dummy: {valor_r2_dummy}')


Filas en Mayo: 2654
Filas en Junio: 2654


--- Cabecera Julio ---


,Producto,Cantidad
69004,CODO DE LAMINA 90 X 4,7.0



--- Cabecera Junio ---


,Producto,Cantidad
0,CODO DE LAMINA 90 X 4,0.0



Error cuadratico del modelo dummy: 9.537213063520435
Error absoluto del modelo dummy: 2.121028635928796
Valor R2 del modelo dummy: -0.7915396732183642


# 6. Prueba de diferentes modelos

In [ ]:
# funcion de evaluacion para modelos:
def evaluar(respuestas,prediccion):
  print(f'Error cuadratico: {np.sqrt(mean_squared_error(respuestas,prediccion))}')
  print(f'Error absoluto: {mean_absolute_error(respuestas,prediccion)}')
  print(f'Valor R2: {r2_score(respuestas,prediccion)}')
  print('-'*15)

In [ ]:
# Modelo 1: Random Forest Regressor
estimators_1 = [25,50,100,200]
for e in estimators_1:
  modelo_1 = RandomForestRegressor(n_estimators=100, random_state=1)
  start = time.time()
  modelo_1.fit(x_train,y_train)
  end = time.time()
  print(f'Tiempo de entrenamiento para {e} estimadores: {end-start}')
  print()
  prediccion_1 = modelo_1.predict(x_test)
  evaluar(y_test,prediccion_1)

Tiempo de entrenamiento para 25 estimadores: 11.303372621536255

Error cuadratico: 6.419826692222646
Error absoluto: 1.9591904203296473
Valor R2: 0.1882354749579621
---------------
Tiempo de entrenamiento para 50 estimadores: 11.183925867080688

Error cuadratico: 6.419826692222646
Error absoluto: 1.9591904203296473
Valor R2: 0.1882354749579621
---------------
Tiempo de entrenamiento para 100 estimadores: 13.531577825546265

Error cuadratico: 6.419826692222646
Error absoluto: 1.9591904203296473
Valor R2: 0.1882354749579621
---------------
Tiempo de entrenamiento para 200 estimadores: 10.378961324691772

Error cuadratico: 6.419826692222646
Error absoluto: 1.9591904203296473
Valor R2: 0.1882354749579621
---------------


In [ ]:
# Modelo 2: Cat Boost Regressor
iteraciones_1 = [100,200,500,1000]
for i in iteraciones_1:
  modelo_2 = CatBoostRegressor(iterations=1,loss_function='RMSE',random_state=1)
  start = time.time()
  modelo_2.fit(x_train,y_train,verbose=10)
  end = time.time()
  print(f'Tiempo de entrenamiento para {i} iteraciones: {end-start}')
  print()
  prediccion_2 = modelo_2.predict(x_test)
  evaluar(y_test,prediccion_2)
# Este modelo mejoro su error cuadratico, aumento el error absoluto en 0.4, pero tambien mejoro el valor R2 en 0.3 en comparacion con RandomForestRegressor

Learning rate set to 0.5
0:	learn: 8.1648356	total: 61.4ms	remaining: 0us
Tiempo de entrenamiento para 100 iteraciones: 0.1477196216583252

Error cuadratico: 5.5054552372107946
Error absoluto: 2.301178952128545
Valor R2: 0.40300604747842794
---------------
Learning rate set to 0.5
0:	learn: 8.1648356	total: 11.4ms	remaining: 0us
Tiempo de entrenamiento para 200 iteraciones: 0.07605481147766113

Error cuadratico: 5.5054552372107946
Error absoluto: 2.301178952128545
Valor R2: 0.40300604747842794
---------------
Learning rate set to 0.5
0:	learn: 8.1648356	total: 12ms	remaining: 0us
Tiempo de entrenamiento para 500 iteraciones: 0.0780785083770752

Error cuadratico: 5.5054552372107946
Error absoluto: 2.301178952128545
Valor R2: 0.40300604747842794
---------------
Learning rate set to 0.5
0:	learn: 8.1648356	total: 10.5ms	remaining: 0us
Tiempo de entrenamiento para 1000 iteraciones: 0.07575654983520508

Error cuadratico: 5.5054552372107946
Error absoluto: 2.301178952128545
Valor R2: 0.40300

In [ ]:
# Modelo 3: X Boost Regressor
estimadores_3 = [3,4,5,6]
for e in estimadores_3:
  modelo_3 = XGBRegressor(n_estimators=e,random_state=1,objective='reg:squarederror',max_depth=5)
  start = time.time()
  modelo_3.fit(x_train,y_train)
  end = time.time()
  print(f'Tiempo de entrenamiento para {e} estimadores: {end-start}')
  prediccion_3 = modelo_3.predict(x_test)
  evaluar(y_test,prediccion_3)
# Este modelo maneja mejor las variables, con el mejor puntaje error cuadratico en comparacion con los anteriores:
# Cuadratico = 5.52
# Absoluto = 1.8
# Valor R2 = 0.4

Tiempo de entrenamiento para 3 estimadores: 0.0845022201538086
Error cuadratico: 5.125826455988302
Error absoluto: 2.043896383515362
Valor R2: 0.4824989246757274
---------------
Tiempo de entrenamiento para 4 estimadores: 0.08528375625610352
Error cuadratico: 5.111969666784918
Error absoluto: 1.9612944523699318
Valor R2: 0.4852930928839605
---------------
Tiempo de entrenamiento para 5 estimadores: 0.07324385643005371
Error cuadratico: 5.214072238528613
Error absoluto: 1.9213909295071043
Valor R2: 0.4645270367169917
---------------
Tiempo de entrenamiento para 6 estimadores: 0.06789112091064453
Error cuadratico: 5.2997145736825395
Error absoluto: 1.883494513239866
Valor R2: 0.44679203940445245
---------------


In [ ]:
# Modelo 4: LGBM Regressor
#estimadores_4 = [10,11,12]
#for e in estimadores_4:
modelo_4 = LGBMRegressor(n_estimators=11,metric='rmse',boosting_type='gbdt',random_state=1,objective='regression')
start = time.time()
modelo_4.fit(x_train,y_train)
end = time.time()
print(f'Tiempo de entrenamiento para {e} estimadores: {end-start}')
print()
prediccion_4 = modelo_4.predict(x_test)
evaluar(y_test,prediccion_4)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006906 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1028
[LightGBM] [Info] Number of data points in the train set: 71658, number of used features: 6
[LightGBM] [Info] Start training from score 2.177479
Tiempo de entrenamiento para 6 estimadores: 0.12426137924194336

Error cuadratico: 5.043334895584348
Error absoluto: 1.9866679791390909
Valor R2: 0.4990215143257155
---------------


#**Conclusion**
1. El mejor modelo es (*modelo 4 LGBMRegressor*).
2. Puntajes: Mejor Puntaje de Error Cuadratico Medio ($5.043334895584348$), con un error absoluto de ($1.9866679791390909$) y con un puntaje de R2 de ($0.4990215143257155$). Mientras que para el modelo dummy de referencia las metricas son: Error cuadratico del modelo dummy: ($9.537213063520435$),
Error absoluto del modelo dummy: ($2.121028635928796$),
Valor R2 del modelo dummy: ($-0.7915396732183642$).
3. El modelo alcanza un correcto desempeño con 11 estimadores, ocupando un tiempo de entrenamiento de ($0.1599137783050537$).
4. Las predicciones se basan en un formato de SeriesTiempo con caracteristicas de 3 meses de lag y una media movil suavizada por una ventana de 3 meses.

# 8. Exportacion del modelo

In [ ]:
# modelo_final = LGBMRegressor(n_estimators=11, metric='rmse', boosting_type='gbdt', random_state=1, objective='regression')

# 1. Exportar el Modelo
joblib.dump(modelo_4, 'lgbm_model.joblib')

# 2. Metadata Completa
metadata = {
    "modelo_ganador": "LGBMRegressor",
    "fecha_exportacion": time.strftime("%Y-%m-%d %H:%M:%S"),
    "hiperparametros_optimos": {
        "n_estimators": 11,
        "max_depth": -1, # Default en LGBM
        "learning_rate": 0.1, # Default
        "boosting_type": "gbdt",
        "random_state": 1
    },
    "ingenieria_caracteristicas": {
        "lags_usados": "1, 2, 3 meses",
        "variables_ciclicas": "Mes (seno/coseno)",
        "otras_features": "Media móvil suavizada (ventana de 3 meses)",
        "variables_eliminadas_final": ['Fecha','Producto','Cantidad','Year','Month']
    },
    "rendimiento_en_test_agosto_2025": {
        "RMSE": 5.0433,
        "MAE": 1.9866,
        "R2": 0.4990,
        "tiempo_entrenamiento_s": 0.1599
    }
}

with open('model_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=4)

In [ ]:
# Exportar prueba para interfaz streamlit, solo como muestra:
agosto = x_test.copy()
agosto = pd.concat([productos_en_test,agosto],axis=1)
display(agosto.head(5))
agosto.to_csv('x_test_agosto.csv',index=False)

,Producto,Cantidad_lag_1,Cantidad_lag_2,Cantidad_lag_3,Rolling_mean,Mes_seno,Mes_coseno
0,CODO DE LAMINA 90 X 4,7.0,3.0,3.0,4.333333,-0.866025,-0.5
1,DUCTO FLEX. MIBER 4 X 2.40 DFL-A4,2.0,2.0,2.0,2.000000,-0.866025,-0.5
2,LAVADERO C/PILETA DE GRANITO INF. 72 X 62CM,0.0,2.0,1.0,1.000000,-0.866025,-0.5
3,BRIDA PARA TONEL 3/4,0.0,0.0,0.0,0.000000,-0.866025,-0.5
4,TUERCA PLANA P/LAVABO,4.0,0.0,0.0,1.333333,-0.866025,-0.5
